Make sure to use `sagemaker==2.93.0`

In [1]:
!pwd

/home/ec2-user/SageMaker/code-repo


In [2]:
! pip install --quiet sagemaker==2.93.0

In [3]:
import boto3
import sagemaker
import sagemaker.session
from sagemaker.workflow.pipeline_context import PipelineSession

In [4]:
sagemaker.__version__

'2.93.0'

In [5]:
region = boto3.Session().region_name
role = sagemaker.get_execution_role()
boto_session = boto3.Session()
sagemaker_session = sagemaker.Session(boto_session=boto_session)
default_bucket = sagemaker_session.default_bucket()

In [6]:
def get_sagemaker_client(region):
    """Gets the sagemaker client.

    Args:
        region: the aws region to start the session
        default_bucket: the bucket to use for storing the artifacts

    Returns:
        `sagemaker.session.Session instance
    """
    boto_session = boto3.Session(region_name=region)
    sagemaker_client = boto_session.client("sagemaker")
    return sagemaker_client


def get_session(region, default_bucket):
    """Gets the sagemaker session based on the region.

    Args:
        region: the aws region to start the session
        default_bucket: the bucket to use for storing the artifacts

    Returns:
        `sagemaker.session.Session instance
    """

    boto_session = boto3.Session(region_name=region)

    sagemaker_client = boto_session.client("sagemaker")
    runtime_client = boto_session.client("sagemaker-runtime")
    return sagemaker.session.Session(
        boto_session=boto_session,
        sagemaker_client=sagemaker_client,
        sagemaker_runtime_client=runtime_client,
        default_bucket=default_bucket,
    )


def get_pipeline_session(region, default_bucket):
    """Gets the pipeline session based on the region.

    Args:
        region: the aws region to start the session
        default_bucket: the bucket to use for storing the artifacts

    Returns:
        PipelineSession instance
    """

    boto_session = boto3.Session(region_name=region)
    sagemaker_client = boto_session.client("sagemaker")

    return PipelineSession(
        boto_session=boto_session,
        sagemaker_client=sagemaker_client,
        default_bucket=default_bucket,
    )

In [7]:
sagemaker_session = get_session(region, default_bucket)
role = sagemaker.session.get_execution_role(sagemaker_session)
pipeline_session = get_pipeline_session(region, default_bucket)

In [8]:
base_job_name = "pipeline-scene"

In [9]:
from sagemaker.workflow.parameters import (
    ParameterFloat,
    ParameterInteger,
    ParameterString,
)

## Processing Step

In [14]:
from sagemaker.processing import FrameworkProcessor, ProcessingInput, ProcessingOutput
from sagemaker.sklearn import SKLearn

In [ ]:
dvc_repo_url = ParameterString(
    name="DVCRepoURL", default_value="codecommit::ap-south-1://sagemaker-scene"
)
dvc_branch = ParameterString(name="DVCBranch", default_value="pipeline-processed-dataset")

In [ ]:
processor = FrameworkProcessor(
    estimator_cls=SKLearn,
    framework_version="0.23-1",
    # instance_type="ml.m5.xlarge",
    instance_type="local",
    instance_count=1,
    image_uri="441249477288.dkr.ecr.ap-south-1.amazonaws.com/sagemaker-training:latest",
    base_job_name=f"{base_job_name}/preprocess-scene-dataset",
    sagemaker_session=pipeline_session,
    role=role,
    env={
        # "DVC_REPO_URL": dvc_repo_url,
        # "DVC_BRANCH": dvc_branch,
        "DVC_REPO_URL": "codecommit::ap-south-1://sagemaker-scene",
        "DVC_BRANCH": "pipeline-processed-dataset",
        "GIT_USER": "gokul-pv",
        "GIT_EMAIL": "25975535+gokul-pv@users.noreply.github.com",
    },
)

In [ ]:
from sagemaker.workflow.steps import ProcessingStep

In [ ]:
input_dataset = ParameterString(
    name="InputDatasetZip",
    default_value="s3://sagemaker-ap-south-1-441249477288/dataset/intel.zip",
)
annotated_meta = ParameterString(
    name="AnnotationMetaData", default_value="s3://sagemaker-ap-south-1-441249477288/annotations/"
)
annotated_data = ParameterString(
    name="AnnotatedData", default_value="s3://sagemaker-ap-south-1-441249477288/Predictions/"
)

In [ ]:
processing_step_args = processor.run(
    code="pipelines/preprocess.py",
    source_dir="sagemaker-kidu",
    inputs=[
        ProcessingInput(
            input_name="data",
            # source=input_dataset,
            source="s3://sagemaker-ap-south-1-441249477288/dataset/intel.zip",
            destination="/opt/ml/processing/input",
        ),
        ProcessingInput(
            input_name="annotated_meta",
            # source=annotated_meta,
            source="s3://sagemaker-ap-south-1-441249477288/old-data/label-studio/",
            destination="/opt/ml/processing/input/annotations/meta",
        ),
        ProcessingInput(
            input_name="annotated_data",
            # source=annotated_data,
            source="s3://sagemaker-ap-south-1-441249477288/old-data/prediction/",
            destination="/opt/ml/processing/input/annotations/data",
        ),
    ],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/dataset/train"),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/dataset/test"),
    ],
)

In [ ]:
from sagemaker.workflow.steps import CacheConfig

cache_config = CacheConfig(enable_caching=True, expire_after="30d")

In [ ]:
step_process = ProcessingStep(
    name="PreprocessSceneClassifierDataset",
    step_args=processing_step_args,
    cache_config=cache_config,
)

In [ ]:
step_process

## Train Step

In [10]:
from sagemaker.debugger import TensorBoardOutputConfig
from sagemaker.pytorch import PyTorch
from sagemaker.workflow.steps import ProcessingStep, TrainingStep

In [11]:
tensorboard_output_config = TensorBoardOutputConfig(
    s3_output_path=f"s3://{default_bucket}/sagemaker-scene-logs-pipeline",
    container_local_output_path="/opt/ml/output/tensorboard",
)

In [12]:
model_name = ParameterString(name="Model", default_value="spnasnet_100")
batch_size = ParameterString(name="Batch_Size", default_value="128")
optimizer = ParameterString(name="Optimizer", default_value="torch.optim.Adam")
learning_rate = ParameterString(name="Learning_Rate", default_value="0.001")
use_augmentation_pipeline = ParameterString(name="Augmentation_Pipeline", default_value="1")
# train_data_s3 = ParameterString(name="Train_Dataset_S3", default_value=step_process.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri)
# test_data_s3 = ParameterString(name="Test_Dataset_S3", default_value=step_process.properties.ProcessingOutputConfig.Outputs["test"].S3Output.S3Uri)

In [20]:
pt_estimator = PyTorch(
    base_job_name=f"{base_job_name}/training-scene-pipeline",
    image_uri="441249477288.dkr.ecr.ap-south-1.amazonaws.com/sagemaker-training:latest",
    source_dir="sagemaker-kidu",
    entry_point="pipelines/train_pipeline.py",
    # sagemaker_session=pipeline_session,
    role=role,
    instance_count=1,
    # instance_type="ml.g4dn.xlarge",
    instance_type="local",
    tensorboard_output_config=tensorboard_output_config,
    # use_spot_instances=True,
    max_wait=1800,
    max_run=1500,
    environment={
        "GIT_USER": "gokul-pv",
        "GIT_EMAIL": "25975535+gokul-pv@users.noreply.github.com",
        # "MODEL": model_name,
        # "BATCH_SIZE": batch_size,
        # "OPTIMIZER": optimizer,
        # "LR": learning_rate,
        # "AUGMENTATION": use_augmentation_pipeline
        "MODEL": "spnasnet_100",
        "BATCH_SIZE": "16",
        "OPTIMIZER": "torch.optim.Adam",
        "LR": "0.0005",
        "AUGMENTATION": "1",
    },
)

In [21]:
from sagemaker.inputs import TrainingInput

In [ ]:
estimator_step_args = pt_estimator.fit(
    {
        "train": "s3://sagemaker-ap-south-1-441249477288/pipeline-scene/preprocess-scene-dataset-2023-01-29-14-40-58-863/output/train/",
        "test": "s3://sagemaker-ap-south-1-441249477288/pipeline-scene/preprocess-scene-dataset-2023-01-29-14-40-58-863/output/test/",
        # 'train': TrainingInput(s3_data=step_process.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri),
        # 'test': TrainingInput(s3_data=step_process.properties.ProcessingOutputConfig.Outputs["test"].S3Output.S3Uri)
    }
)

Creating 93vbopgk1t-algo-1-tnbe6 ... 
Creating 93vbopgk1t-algo-1-tnbe6 ... done
Attaching to 93vbopgk1t-algo-1-tnbe6
93vbopgk1t-algo-1-tnbe6 | sed: can't read changehostname.c: No such file or directory
93vbopgk1t-algo-1-tnbe6 | gcc: error: changehostname.c: No such file or directory
93vbopgk1t-algo-1-tnbe6 | gcc: fatal error: no input files
93vbopgk1t-algo-1-tnbe6 | compilation terminated.
93vbopgk1t-algo-1-tnbe6 | gcc: error: changehostname.o: No such file or directory
93vbopgk1t-algo-1-tnbe6 | ERROR: ld.so: object '/libchangehostname.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
93vbopgk1t-algo-1-tnbe6 | 2023-02-05 14:40:28,090 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
93vbopgk1t-algo-1-tnbe6 | 2023-02-05 14:40:28,093 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
93vbopgk1t-algo-1-tnbe6 | 2023-02-05 14:40:28,095 sagemaker-training-toolkit INFO     No Neurons de

In [ ]:
step_train = TrainingStep(
    name="TrainSceneClassifier", step_args=estimator_step_args, cache_config=cache_config
)

In [ ]:
step_train

## Eval Step

In [10]:
from sagemaker.pytorch.processing import PyTorchProcessor

In [12]:
pytorch_processor = PyTorchProcessor(
    image_uri="441249477288.dkr.ecr.ap-south-1.amazonaws.com/sagemaker-training:latest",
    framework_version="1.11.0",
    py_version="py38",
    role=role,
    sagemaker_session=pipeline_session,
    # instance_type="ml.m5.4xlarge",
    instance_type="local",
    instance_count=1,
    base_job_name=f"{base_job_name}/eval-scene-classifier",
    env={
        # "BATCH_SIZE": batch_size,
        # "MODEL": model_name,
        "BATCH_SIZE": "16",
        "MODEL": "spnasnet_100",
    },
)

In [15]:
eval_step_args = pytorch_processor.run(
    code="pipelines/evaluate.py",
    source_dir="sagemaker-kidu",
    inputs=[
        ProcessingInput(
            # source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
            source="s3://sagemaker-ap-south-1-441249477288/pipelines-7mypdpqubax8-TrainSceneClassifier-u9XudMzGTd/output/model.tar.gz",
            destination="/opt/ml/processing/model",
        ),
        ProcessingInput(
            # source=step_process.properties.ProcessingOutputConfig.Outputs["test"].S3Output.S3Uri,
            source="s3://sagemaker-ap-south-1-441249477288/pipeline-scene/preprocess-scene-dataset-2023-01-29-14-40-58-863/output/test/",
            destination="/opt/ml/processing/test",
        ),
        ProcessingInput(
            # source=step_process.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri,
            source="s3://sagemaker-ap-south-1-441249477288/pipeline-scene/preprocess-scene-dataset-2023-01-29-14-40-58-863/output/test/",
            destination="/opt/ml/processing/train",
        ),
        ProcessingInput(
            source="s3://sagemaker-ap-south-1-441249477288/old-data/prediction/",
            destination="/opt/ml/processing/prediction",
        ),
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
    ],
)


Job Name:  pipeline-scene/eval-scene-classifier-2023-02-05-16-17-46-293
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-south-1-441249477288/pipelines-7mypdpqubax8-TrainSceneClassifier-u9XudMzGTd/output/model.tar.gz', 'LocalPath': '/opt/ml/processing/model', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'input-2', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-south-1-441249477288/pipeline-scene/preprocess-scene-dataset-2023-01-29-14-40-58-863/output/test/', 'LocalPath': '/opt/ml/processing/test', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'input-3', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-south-1-441249477288/pipeline-scene/preprocess-scene-dataset-2023-01-29-14-40-58-863/output/test/', 'LocalPath': '/opt/ml/processi

RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmphhcm0lra/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit']

In [ ]:
from sagemaker.workflow.properties import PropertyFile

In [ ]:
evaluation_report = PropertyFile(
    name="SceneClassifierEvaluationReport",
    output_name="evaluation",
    path="evaluation.json",
)
step_eval = ProcessingStep(
    name="EvaluateSceneClassifierModel",
    step_args=eval_step_args,
    property_files=[evaluation_report],
)

In [ ]:
step_eval

## Model Metrics

In [ ]:
from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.workflow.properties import PropertyFile

In [ ]:
model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri="{}/evaluation.json".format(
            step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
        ),
        # s3_uri="s3://sagemaker-ap-south-1-006547668672/eval-flower-classifier-model-2022-12-07-19-40-04-608/output/evaluation/evaluation.json",
        content_type="application/json",
    )
)

## Register Model Step (Conditional)

In [32]:
from sagemaker.deserializers import JSONDeserializer
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker.pytorch import PyTorchModel
from sagemaker.serializers import DataSerializer, JSONSerializer, NumpySerializer
from sagemaker.workflow.model_step import ModelStep

In [33]:
model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)

In [34]:
model_package_group_name = "SceneClassifierModelGroup"

In [35]:
# model = PyTorchModel(
#     entry_point="pipelines/infer.py",
#     source_dir="sagemaker-kidu",
#     image_uri = "441249477288.dkr.ecr.ap-south-1.amazonaws.com/sagemaker-inference",
#     sagemaker_session=pipeline_session,
#     role=role,
#     model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
#     # model_data="s3://sagemaker-ap-south-1-006547668672/training-flower-pipeline-2022-12-07-03-20-21-157/output/model.tar.gz",
#     framework_version="1.11.0",
# )

In [36]:
# !aws ecr get-login-password --region ap-south-1 | docker login --username AWS --password-stdin 441249477288.dkr.ecr.ap-south-1.amazonaws.com

In [37]:
model = Model(
    image_uri="441249477288.dkr.ecr.ap-south-1.amazonaws.com/sagemaker-inference:latest",
    model_data="s3://sagemaker-project-p-yuysbbvtdpyo/pipelines-w7h248umyt5i-TrainIntelClassifier-AoArHjW6QS/output/model.tar.gz",
    role=role,
    # sagemaker_session=pipeline_session
    predictor_cls=Predictor,
)

In [38]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.t2.medium",
    serializer=DataSerializer(),
    deserializer=JSONDeserializer(),
)

-------!

In [ ]:
# predictor_new = Predictor(
#     endpoint_name="sagemaker-inference-2023-02-03-12-33-35-382",
#     sagemaker_session=sagemaker_session,
#     serializer=DataSerializer(),
#     deserializer=JSONDeserializer(),
# )

In [39]:
out = predictor.predict("/home/ec2-user/SageMaker/5.jpg")

In [40]:
out

{'buildings': 0.9999991655349731,
 'sea': 7.872914125073294e-07,
 'street': 1.1917365982938577e-09,
 'glacier': 7.40345840188894e-10,
 'mountain': 2.2310342462361632e-10,
 'forest': 4.979721929948555e-12}

In [ ]:
model_step_args = model.register(
    content_types=["file-path/raw-bytes"],
    response_types=["application/json"],
    inference_instances=["ml.t2.medium", "ml.t2.large"],
    transform_instances=["ml.m4.xlarge"],
    model_package_group_name=model_package_group_name,
    # approval_status=model_approval_status,
    approval_status="PendingManualApproval",
    model_metrics=model_metrics,
)

In [ ]:
step_register = ModelStep(
    name="RegisterSceneClassifierModel",
    step_args=model_step_args,
)

In [ ]:
step_register

In [ ]:
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.functions import JsonGet

In [ ]:
cond_gte = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step_name=step_eval.name,
        property_file=evaluation_report,
        json_path="multiclass_classification_metrics.accuracy.value",
    ),
    right=0.6,
)

step_cond = ConditionStep(
    name="CheckAccuracySceneClassifierEvaluation",
    conditions=[cond_gte],
    if_steps=[step_register],
    else_steps=[],
)

## Pipeline

In [ ]:
from sagemaker.workflow.pipeline import Pipeline

In [ ]:
pipeline_name = "PyTorchSceneClassifier"

In [ ]:
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        dvc_repo_url,
        dvc_branch,
        input_dataset,
        model_approval_status,
        model_name,
        batch_size,
        optimizer,
        learning_rate,
    ],
    steps=[step_process, step_train, step_eval, step_cond],
    sagemaker_session=pipeline_session,
    # sagemaker_session=local_pipeline_session,
)

In [ ]:
upsert_response = pipeline.upsert(role_arn=role, description="testing pytorch scene pipeline")

In [ ]:
execution = pipeline.start()

In [ ]:
execution.list_steps()

In [ ]:
execution.describe()